原文：http://nbviewer.jupyter.org/github/jmsteinw/Notebooks/blob/master/NLP_Movies.ipynb#

In [2]:
import re
from bs4 import BeautifulSoup

In [4]:
def review_to_wordlist(review):
    """
    将每个评论转换为词列表
    """
    # remove HTML
    review_text = BeautifulSoup(review).get_text()
    # 不是字母的字符都替换为空格
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    # 都变成小写，并根据空格分词
    words = review_text.lower().split()
    return (words)


 读入数据

In [10]:
import os
import pandas as pd

In [17]:
datafile = os.path.join('data', 'labeledTrainData.tsv')
train = pd.read_csv(datafile, header=0, delimiter='\t', quoting=3)
datafile = os.path.join('data', 'testData.tsv')
test = pd.read_csv(datafile, header=0, delimiter="\t", quoting=3)


In [22]:
type(train)

pandas.core.frame.DataFrame

In [19]:
# 得到标签，1表示积极，0表示消极
y_train=train['sentiment']

 清理训练数据和测试数据

In [21]:
traindata = []
for i in range(0, len(train['review'])):
    traindata.append(' '.join(review_to_wordlist(train['review'][i])))
testdata = []
for i in range(0, len(test['review'])):
    testdata.append(" ".join(review_to_wordlist(test['review'][i])))


d:\ProgramData\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file d:\ProgramData\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [26]:
traindata[0]

'with all this stuff going down at the moment with mj i ve started listening to his music watching the odd documentary here and there watched the wiz and watched moonwalker again maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent moonwalker is part biography part feature film which i remember going to see at the cinema when it was originally released some of it has subtle messages about mj s feeling towards the press and also the obvious message of drugs are bad m kay visually impressive but of course this is all about michael jackson so unless you remotely like mj in anyway then you are going to hate this and find it boring some may call mj an egotist for consenting to the making of this movie but mj and most of his fans would say that he made it for the fans which if true is really nice of him the actual feature film bit when it finally starts is only on for minutes or s

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer as TFIV

tfv = TFIV(min_df=3, max_features=None, strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
           ngram_range=(1, 2), use_idf=1, smooth_idf=1, sublinear_tf=1, stop_words='english')

X_all = traindata + testdata  # 训练和测试数据都用来训练TFIDF向量
lentrain = len(traindata)

tfv.fit(X_all)
X_all = tfv.transform(X_all)

# 将训练集合测试集分开
X = X_all[:lentrain]
X_test = X_all[lentrain:]


In [29]:
X.shape  # 25000个训练样本，每个样本有309819维特征

(25000, 309819)

 1、Logistic Regression

In [46]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.grid_search import GridSearchCV

grid_values = {'C': [30]}  # Decide which settings you want for the grid search. 

model_LR = GridSearchCV(LR(dual=True, random_state=0),
                        grid_values, scoring='roc_auc', cv=20)
# Try to set the scoring on what the contest is asking for. 
# The contest says scoring is for area under the ROC curve, so use this.

model_LR.fit(X, y_train)  # Fit the model.


GridSearchCV(cv=20, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [30]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [47]:
model_LR.grid_scores_

[mean: 0.96461, std: 0.00490, params: {'C': 30}]

In [48]:
model_LR.best_estimator_

LogisticRegression(C=30, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

 2、Multinomial Naive Bayes

In [50]:
from sklearn.naive_bayes import MultinomialNB as MNB

In [51]:
model_NB = MNB()
model_NB.fit(X, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [52]:
from sklearn.cross_validation import cross_val_score
import numpy as np

print("20 Fold CV Score for Multinomial Naive Bayes:",
      np.mean(cross_val_score(model_NB, X, y_train, cv=20, scoring='roc_auc')))


20 Fold CV Score for Multinomial Naive Bayes: 0.94963712


 3、SGD classifier

In [53]:
from sklearn.linear_model import SGDClassifier as SGD

In [54]:
sgd_params={'alpha': [0.00006, 0.00007, 0.00008, 0.0001, 0.0005]}
model_SGD=GridSearchCV(SGD(random_state=0,shuffle=True,loss='modified_huber'),sgd_params,scoring='roc_auc',cv=20)
model_SGD.fit(X,y_train)

GridSearchCV(cv=20, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='modified_huber', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=0, shuffle=True, verbose=0,
       warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'alpha': [6e-05, 7e-05, 8e-05, 0.0001, 0.0005]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [55]:
model_SGD.grid_scores_

[mean: 0.96479, std: 0.00485, params: {'alpha': 6e-05},
 mean: 0.96486, std: 0.00481, params: {'alpha': 7e-05},
 mean: 0.96488, std: 0.00481, params: {'alpha': 8e-05},
 mean: 0.96479, std: 0.00480, params: {'alpha': 0.0001},
 mean: 0.95871, std: 0.00484, params: {'alpha': 0.0005}]

In [61]:
LR_result = model_LR.predict_proba(X_test)[:, 1]
LR_output = pd.DataFrame(data={'id': test['id'], 'sentiment': LR_result})
LR_output.to_csv('Logistic_Reg_Proj2.csv', index=False, quoting=3) # Get the .csv file we will submit to Kaggle.

In [57]:
LR_result

array([ 0.99692255,  0.00761538,  0.55690482, ...,  0.3559734 ,
        0.97681579,  0.71244144])

In [64]:
MNB_result = model_NB.predict_proba(X_test)[:, 1]
MNB_output = pd.DataFrame(data={'id': test['id'], 'sentiment': MNB_result})
MNB_output.to_csv('MNB_Proj2.csv', index=False, quoting=3)


In [65]:
SGD_result = model_SGD.predict_proba(X_test)[:, 1]
SGD_output = pd.DataFrame(data={'id': test['id'], 'sentiment': SGD_result})
SGD_output.to_csv('SGD_Proj2.csv', index=False, quoting=3)
